In [1]:
cd /home/karen/Documents/GitHub/Muscle_wasting

/home/karen/Documents/GitHub/Muscle_wasting


/home/karen/Documents/GitHub/Muscle_wasting/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import mirna_scoring.walking_network as wn
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mirna_scoring.mirna_impact as mis
import seaborn as sns
import mirna_scoring.mirna_influence_plots as mi_plot
import mirna_scoring.jupyter_functions as jf

In [3]:
from importlib import reload  # Python 3.4+
import mirna_scoring.mirna_impact as mis

In [5]:
network = nx.read_gpickle(f"network/Networks_pkl/complete_n_tf_mirnas__Sarcopenia_relevant_normalize_cutoff_0.9.pkl")
my_network = mis.mirna_network(network=network)

In [6]:
my_network.quick_get_all_scores(steps=5, sample_size=10, dds_threshold=0, pathway_keywords=['ATP'])

KeyError: 'metadata'

Index(['SOX4', 'TGFBR2', 'MYOM2', 'TGFB2', 'IL1B', 'CD274', 'MECP2', 'SMAD7',
       'ZEB2', 'TLR2',
       ...
       'CRYAB', 'ESR1', 'JUN', 'CFTR', 'AR', 'NFIC', 'HBB', 'ALDOA', 'SP1',
       'STAT5B'],
      dtype='object', length=198)